In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

import wrangle

In [2]:
air = wrangle.clean_air()
air.head(1)

,datetime,Pm1_0,Pm2_5,Pm10,SO2,O3,CO,NO2,alert_triggered,dates,...,AQI_pm2_5,Pm_25_24hr,AQI_pm_25_24hr,AQI_pm10,Pm_10_24hr,AQI_pm10_24hr,unhealthy_alert,sensitive_alert,hypothetical_unhealthy_alert,hypothetical_sensitive_alert
0,2021-05-20 00:01:27,8,6,8,0,0.0,0,0,None,2021-05-20,...,Good,6.704282,Good,Good,8.568287,Good,No Alert,No Alert,No Alert,No Alert


In [3]:
air.SO2.value_counts()

0    106276
1         5
6         3
3         2
2         1
Name: SO2, dtype: int64

In [4]:
air['round_8hr'] = air['datetime']
air['round_8hr'] = air['round_8hr'].dt.round('480min')

In [5]:
air = air.drop(columns=['alert_triggered', 'dates', 'time',
                        'CO_24hr', 'AQI_CO_24hr', 'Pm_25_24hr',
                        'AQI_pm_25_24hr', 'Pm_10_24hr', 
                        'AQI_pm10_24hr', 'datetime'])

In [6]:
air.sample(5)

,Pm1_0,Pm2_5,Pm10,SO2,O3,CO,NO2,hour,weekday,AQI_CO,AQI_pm2_5,AQI_pm10,unhealthy_alert,sensitive_alert,hypothetical_unhealthy_alert,hypothetical_sensitive_alert,round_8hr
18818,0,8,0,0,24.625,0,6,3,6,Good,Good,Good,No Alert,No Alert,O3,O3,2021-05-16 00:00:00
75914,17,20,24,0,0.000,0,0,20,0,Good,Moderate,Good,No Alert,No Alert,No Alert,No Alert,2021-06-22 00:00:00
37952,0,2,3,0,0.000,0,1,13,5,Good,Good,Good,No Alert,No Alert,NO2,NO2,2021-05-29 16:00:00
63223,0,0,0,0,10.630,0,9,8,6,Good,Good,Good,No Alert,No Alert,O3,O3,2021-06-13 08:00:00
38822,0,0,0,0,35.070,0,1,15,5,Good,Good,Good,No Alert,No Alert,O3,O3,2021-05-29 16:00:00


In [7]:
average_8hr = air.groupby('round_8hr', as_index=False).mean()

In [8]:
average_8hr['AQI_CO'] = pd.cut(average_8hr.CO, 
                            bins = [-1,4.5,9.5,12.5,15.5,30.5,4000],
                            labels = ['Good', 'Moderate', 
                                      'Unhealthy for Sensitive Groups', "Unhealthy", 
                                      "Very Unhealthy", 'Hazardous'])

In [9]:
average_8hr['AQI_pm2_5'] = pd.cut(average_8hr.Pm2_5, 
                                bins = [-1,12.1,35.5,55.5,150.5,250.5,4000],
                                labels = ['Good', 'Moderate', 
                                          'Unhealthy for Sensitive Groups', "Unhealthy", 
                                          "Very Unhealthy", 'Hazardous'])

In [10]:
average_8hr['AQI_pm10'] = pd.cut(average_8hr.Pm10, 
                                bins = [-1,55,154,255,355,425,4000],
                                labels = ['Good', 'Moderate', 
                                          'Unhealthy for Sensitive Groups', "Unhealthy", 
                                          "Very Unhealthy", 'Hazardous'])

In [11]:
average_8hr['AQI_SO2'] = pd.cut(average_8hr.SO2, 
                                bins = [-1,0.0359,0.0759,0.1859,0.3049,0.6049,4000],
                                labels = ['Good', 'Moderate', 
                                          'Unhealthy for Sensitive Groups', "Unhealthy", 
                                          "Very Unhealthy", 'Hazardous'])

In [12]:
average_8hr['AQI_NO2'] = pd.cut(average_8hr.NO2, 
                                bins = [-1,0.0539,0.1009,0.3609,0.6499,1.2499,4000],
                                labels = ['Good', 'Moderate', 
                                          'Unhealthy for Sensitive Groups', "Unhealthy", 
                                          "Very Unhealthy", 'Hazardous'])

In [13]:
average_8hr['AQI_O3'] = pd.cut(average_8hr.O3, 
                                bins = [-1,0.0549,0.0709,0.0859,0.1059,0.2009,4000],
                                labels = ['Good', 'Moderate', 
                                          'Unhealthy for Sensitive Groups', "Unhealthy", 
                                          "Very Unhealthy", 'Hazardous'])

In [14]:
average_8hr.sample(5)

,round_8hr,Pm1_0,Pm2_5,Pm10,SO2,O3,CO,NO2,hour,weekday,AQI_CO,AQI_pm2_5,AQI_pm10,AQI_SO2,AQI_NO2,AQI_O3
143,2021-06-06 16:00:00,5.003466,5.982669,7.079723,0.000000,5.581196,0.225303,1.183709,15.507799,6.0,Good,Good,Good,Good,Very Unhealthy,Hazardous
44,2021-05-04 16:00:00,1.134783,3.965217,71.921739,0.000000,18.244076,0.082609,2.600000,15.373913,1.0,Good,Good,Moderate,Good,Hazardous,Hazardous
101,2021-05-23 16:00:00,1.663778,2.287695,2.876950,0.000000,6.404281,0.034662,0.369151,15.507799,6.0,Good,Good,Good,Good,Unhealthy,Hazardous
41,2021-05-03 16:00:00,22.982759,33.987069,27.620690,0.025862,22.292403,0.228448,3.237069,15.370690,0.0,Good,Moderate,Good,Good,Hazardous,Hazardous
13,2021-04-24 08:00:00,2.588652,3.276596,118.531915,0.000000,0.001418,0.588652,0.304965,7.446809,5.0,Good,Good,Moderate,Good,Unhealthy for Sensitive Groups,Good


In [15]:
average_8hr = average_8hr.rename(columns={"round_8hr": "datetime"})

In [16]:
average_8hr.sample(1)

,datetime,Pm1_0,Pm2_5,Pm10,SO2,O3,CO,NO2,hour,weekday,AQI_CO,AQI_pm2_5,AQI_pm10,AQI_SO2,AQI_NO2,AQI_O3
220,2021-07-02 08:00:00,4.41913,281.083478,6.834783,0.0,2.424296,0.086957,1.728696,7.506087,4.0,Good,Hazardous,Good,Good,Hazardous,Hazardous


In [17]:
average_8hr.AQI_NO2.value_counts()

Hazardous                         152
Unhealthy for Sensitive Groups     39
Very Unhealthy                     30
Unhealthy                          19
Moderate                            1
Good                                0
Name: AQI_NO2, dtype: int64

In [18]:
average_8hr.AQI_O3.value_counts()

Hazardous                         206
Good                               34
Unhealthy                           1
Very Unhealthy                      0
Unhealthy for Sensitive Groups      0
Moderate                            0
Name: AQI_O3, dtype: int64

In [19]:
average_8hr.AQI_SO2.value_counts()

Good                              239
Unhealthy for Sensitive Groups      1
Moderate                            1
Hazardous                           0
Very Unhealthy                      0
Unhealthy                           0
Name: AQI_SO2, dtype: int64

In [20]:
air = wrangle.clean_air()

def air_8hr_avg(air):
    '''Takes in air df and creates every 8 hour averages'''
    # duplicate datetime column
    air['round_8hr'] = air['datetime']
    # round to every 8 hours
    air['round_8hr'] = air['round_8hr'].dt.round('480min')
    # create mew df based on the rouned time
    average_8hr = air.groupby('round_8hr', as_index=False).mean()
    # Create AQI for CO
    average_8hr['AQI_CO'] = pd.cut(average_8hr.CO, 
                                bins = [-1,4.5,9.5,12.5,15.5,30.5,4000],
                                labels = ['Good', 'Moderate', 
                                          'Unhealthy for Sensitive Groups', "Unhealthy", 
                                          "Very Unhealthy", 'Hazardous'])
    # Create AQI for Pm2_5
    average_8hr['AQI_pm2_5'] = pd.cut(average_8hr.Pm2_5, 
                                    bins = [-1,12.1,35.5,55.5,150.5,250.5,4000],
                                    labels = ['Good', 'Moderate', 
                                              'Unhealthy for Sensitive Groups', "Unhealthy", 
                                              "Very Unhealthy", 'Hazardous'])
    # Create AQI for Pm10
    average_8hr['AQI_pm10'] = pd.cut(average_8hr.Pm10, 
                                    bins = [-1,55,154,255,355,425,4000],
                                    labels = ['Good', 'Moderate', 
                                              'Unhealthy for Sensitive Groups', "Unhealthy", 
                                              "Very Unhealthy", 'Hazardous'])
    # create AQI for SO2
    average_8hr['AQI_SO2'] = pd.cut(average_8hr.SO2, 
                                    bins = [-1,0.0359,0.0759,0.1859,0.3049,0.6049,4000],
                                    labels = ['Good', 'Moderate', 
                                              'Unhealthy for Sensitive Groups', "Unhealthy", 
                                              "Very Unhealthy", 'Hazardous'])
    # create AQI for NO2
    average_8hr['AQI_NO2'] = pd.cut(average_8hr.NO2, 
                                    bins = [-1,0.0539,0.1009,0.3609,0.6499,1.2499,4000],
                                    labels = ['Good', 'Moderate', 
                                              'Unhealthy for Sensitive Groups', "Unhealthy", 
                                              "Very Unhealthy", 'Hazardous'])
    # create AQI for O3
    average_8hr['AQI_O3'] = pd.cut(average_8hr.O3, 
                                    bins = [-1,0.0549,0.0709,0.0859,0.1059,0.2009,4000],
                                    labels = ['Good', 'Moderate', 
                                              'Unhealthy for Sensitive Groups', "Unhealthy", 
                                              "Very Unhealthy", 'Hazardous'])
    # rename the new date time column
    average_8hr = average_8hr.rename(columns={"round_8hr": "datetime"})
    # Create air alerts
    def unhealthy_air_alert(c):
        if c['Pm2_5'] > 55.4:
            return 'Pm2_5'
        elif c['Pm10'] > 254.9:
            return 'Pm10'
        elif c['CO'] > 12.4:
            return 'CO'
        elif c['SO2'] > 0.1859:
            return 'SO2'
        elif c['O3'] > 0.1649:
            return 'O3'
        elif c['NO2'] > 0.3609:
            return 'NO2'
        else:
            return 'No Alert'
    # Apply alert
    average_8hr['unhealthy_alert'] = average_8hr.apply(unhealthy_air_alert, axis=1)
    # create sensitive HYPOTHTICAL Alert system
        # hypothetical alerts are based on IF everything is reading in PPM
    def sensitive_air_alert(c):
        if c['Pm2_5'] > 34.4:
            return 'Pm2_5'
        elif c['Pm10'] > 154.9:
            return 'Pm10'
        elif c['CO'] > 9.4:
            return 'CO'
        elif c['SO2'] > 0.0759:
            return 'SO2'
        elif c['O3'] > 0.124:
            return 'O3'
        elif c['NO2'] > 0.1009:
            return 'NO2'
        else:
            return 'No Alert'
    # apply alert
    average_8hr['sensitive_alert'] = average_8hr.apply(sensitive_air_alert, axis=1)
    return average_8hr

average_8hr = air_8hr_avg(air)
average_8hr.head()

,datetime,Pm1_0,Pm2_5,Pm10,SO2,O3,CO,NO2,hour,weekday,...,Pm_25_24hr,Pm_10_24hr,AQI_CO,AQI_pm2_5,AQI_pm10,AQI_SO2,AQI_NO2,AQI_O3,unhealthy_alert,sensitive_alert
0,2021-04-20 00:00:00,7.555556,8.375000,10.638889,0.0,0.000000,0.055556,0.305556,1.500000,1.0,...,8.022792,7.324786,Good,Good,Good,Good,Unhealthy for Sensitive Groups,Good,No Alert,NO2
1,2021-04-20 08:00:00,4.423611,10.763889,6.680556,0.0,0.017361,0.486111,0.340278,7.500000,1.0,...,8.022792,7.324786,Good,Good,Good,Good,Unhealthy for Sensitive Groups,Good,No Alert,NO2
2,2021-04-20 16:00:00,4.755556,4.911111,6.244444,0.0,0.002222,0.229630,0.125926,15.266667,1.0,...,8.022792,7.324786,Good,Good,Good,Good,Unhealthy for Sensitive Groups,Good,No Alert,NO2
3,2021-04-21 00:00:00,2.400000,3.053333,3.800000,0.0,0.004000,0.000000,0.133333,1.480000,2.0,...,3.616477,4.460227,Good,Good,Good,Good,Unhealthy for Sensitive Groups,Good,No Alert,NO2
4,2021-04-21 08:00:00,2.715278,3.652778,4.506944,0.0,0.002778,0.173611,0.152778,7.500000,2.0,...,3.616477,4.460227,Good,Good,Good,Good,Unhealthy for Sensitive Groups,Good,No Alert,NO2


In [21]:
air = wrangle.clean_air()

def air_12hr_avg(air):
    '''Takes in air df and creates every 8 hour averages'''
    # duplicate datetime column
    air['round_12hr'] = air['datetime']
    # round to every 8 hours
    air['round_12hr'] = air['round_12hr'].dt.round('720min')
    # create mew df based on the rouned time
    average_12hr = air.groupby('round_12hr', as_index=False).mean()
    # Create AQI for CO
    average_12hr['AQI_CO'] = pd.cut(average_12hr.CO, 
                                bins = [-1,4.5,9.5,12.5,15.5,30.5,4000],
                                labels = ['Good', 'Moderate', 
                                          'Unhealthy for Sensitive Groups', "Unhealthy", 
                                          "Very Unhealthy", 'Hazardous'])
    # Create AQI for Pm2_5
    average_12hr['AQI_pm2_5'] = pd.cut(average_12hr.Pm2_5, 
                                    bins = [-1,12.1,35.5,55.5,150.5,250.5,4000],
                                    labels = ['Good', 'Moderate', 
                                              'Unhealthy for Sensitive Groups', "Unhealthy", 
                                              "Very Unhealthy", 'Hazardous'])
    # Create AQI for Pm10
    average_12hr['AQI_pm10'] = pd.cut(average_12hr.Pm10, 
                                    bins = [-1,55,154,255,355,425,4000],
                                    labels = ['Good', 'Moderate', 
                                              'Unhealthy for Sensitive Groups', "Unhealthy", 
                                              "Very Unhealthy", 'Hazardous'])
    # create AQI for SO2
    average_12hr['AQI_SO2'] = pd.cut(average_12hr.SO2, 
                                    bins = [-1,0.0359,0.0759,0.1859,0.3049,0.6049,4000],
                                    labels = ['Good', 'Moderate', 
                                              'Unhealthy for Sensitive Groups', "Unhealthy", 
                                              "Very Unhealthy", 'Hazardous'])
    # create AQI for NO2
    average_12hr['AQI_NO2'] = pd.cut(average_12hr.NO2, 
                                    bins = [-1,0.0539,0.1009,0.3609,0.6499,1.2499,4000],
                                    labels = ['Good', 'Moderate', 
                                              'Unhealthy for Sensitive Groups', "Unhealthy", 
                                              "Very Unhealthy", 'Hazardous'])
    # create AQI for O3
    average_12hr['AQI_O3'] = pd.cut(average_12hr.O3, 
                                    bins = [-1,0.0549,0.0709,0.0859,0.1059,0.2009,4000],
                                    labels = ['Good', 'Moderate', 
                                              'Unhealthy for Sensitive Groups', "Unhealthy", 
                                              "Very Unhealthy", 'Hazardous'])
    # rename the new date time column
    average_12hr = average_12hr.rename(columns={"round_12hr": "datetime"})
    # Create air alerts
    def unhealthy_air_alert(c):
        if c['Pm2_5'] > 55.4:
            return 'Pm2_5'
        elif c['Pm10'] > 254.9:
            return 'Pm10'
        elif c['CO'] > 12.4:
            return 'CO'
        elif c['SO2'] > 0.1859:
            return 'SO2'
        elif c['O3'] > 0.1649:
            return 'O3'
        elif c['NO2'] > 0.3609:
            return 'NO2'
        else:
            return 'No Alert'
    average_12hr['unhealthy_alert'] = average_12hr.apply(unhealthy_air_alert, axis=1)
    # create sensitive HYPOTHTICAL Alert system
        # hypothetical alerts are based on IF everything is reading in PPM
    def sensitive_air_alert(c):
        if c['Pm2_5'] > 34.4:
            return 'Pm2_5'
        elif c['Pm10'] > 154.9:
            return 'Pm10'
        elif c['CO'] > 9.4:
            return 'CO'
        elif c['SO2'] > 0.0759:
            return 'SO2'
        elif c['O3'] > 0.124:
            return 'O3'
        elif c['NO2'] > 0.1009:
            return 'NO2'
        else:
            return 'No Alert'

    average_12hr['sensitive_alert'] = average_12hr.apply(sensitive_air_alert, axis=1)

    return average_12hr

average_12hr = air_12hr_avg(air)
average_12hr.head()

,datetime,Pm1_0,Pm2_5,Pm10,SO2,O3,CO,NO2,hour,weekday,...,Pm_25_24hr,Pm_10_24hr,AQI_CO,AQI_pm2_5,AQI_pm10,AQI_SO2,AQI_NO2,AQI_O3,unhealthy_alert,sensitive_alert
0,2021-04-20 00:00:00,7.111111,7.888889,9.962963,0.0,0.023148,0.129630,0.277778,2.500000,1.000000,...,8.022792,7.324786,Good,Good,Good,Good,Unhealthy for Sensitive Groups,Good,No Alert,NO2
1,2021-04-20 12:00:00,4.171296,8.421296,6.050926,0.0,0.000926,0.421296,0.250000,11.500000,1.000000,...,8.022792,7.324786,Good,Good,Good,Good,Unhealthy for Sensitive Groups,Good,No Alert,NO2
2,2021-04-21 00:00:00,3.065217,3.449275,4.369565,0.0,0.004348,0.000000,0.101449,5.565217,1.804348,...,4.478582,5.020684,Good,Good,Good,Good,Unhealthy for Sensitive Groups,Good,No Alert,NO2
3,2021-04-21 12:00:00,3.247664,3.976636,4.901869,0.0,0.002336,0.238318,0.158879,11.476636,2.000000,...,3.616477,4.460227,Good,Good,Good,Good,Unhealthy for Sensitive Groups,Good,No Alert,NO2
4,2021-04-22 00:00:00,7.725926,8.355556,10.770370,0.0,0.020000,0.022222,0.259259,5.666667,2.800000,...,15.321023,19.262500,Good,Good,Good,Good,Unhealthy for Sensitive Groups,Good,No Alert,NO2


In [22]:
air = wrangle.clean_air()

def air_24hr_avg(air):
    '''Takes in air df and creates every 8 hour averages'''
    # duplicate datetime column
    air['round_24hr'] = air['datetime']
    # round to every 8 hours
    air['round_24hr'] = air['round_24hr'].dt.round('1440min')
    # create mew df based on the rouned time
    average_24hr = air.groupby('round_24hr', as_index=False).mean()
    # Create AQI for CO
    average_24hr['AQI_CO'] = pd.cut(average_24hr.CO, 
                                bins = [-1,4.5,9.5,12.5,15.5,30.5,4000],
                                labels = ['Good', 'Moderate', 
                                          'Unhealthy for Sensitive Groups', "Unhealthy", 
                                          "Very Unhealthy", 'Hazardous'])
    # Create AQI for Pm2_5
    average_24hr['AQI_pm2_5'] = pd.cut(average_24hr.Pm2_5, 
                                    bins = [-1,12.1,35.5,55.5,150.5,250.5,4000],
                                    labels = ['Good', 'Moderate', 
                                              'Unhealthy for Sensitive Groups', "Unhealthy", 
                                              "Very Unhealthy", 'Hazardous'])
    # Create AQI for Pm10
    average_24hr['AQI_pm10'] = pd.cut(average_24hr.Pm10, 
                                    bins = [-1,55,154,255,355,425,4000],
                                    labels = ['Good', 'Moderate', 
                                              'Unhealthy for Sensitive Groups', "Unhealthy", 
                                              "Very Unhealthy", 'Hazardous'])
    # create AQI for SO2
    average_24hr['AQI_SO2'] = pd.cut(average_24hr.SO2, 
                                    bins = [-1,0.0359,0.0759,0.1859,0.3049,0.6049,4000],
                                    labels = ['Good', 'Moderate', 
                                              'Unhealthy for Sensitive Groups', "Unhealthy", 
                                              "Very Unhealthy", 'Hazardous'])
    # create AQI for NO2
    average_24hr['AQI_NO2'] = pd.cut(average_24hr.NO2, 
                                    bins = [-1,0.0539,0.1009,0.3609,0.6499,1.2499,4000],
                                    labels = ['Good', 'Moderate', 
                                              'Unhealthy for Sensitive Groups', "Unhealthy", 
                                              "Very Unhealthy", 'Hazardous'])
    # create AQI for O3
    average_24hr['AQI_O3'] = pd.cut(average_24hr.O3, 
                                    bins = [-1,0.0549,0.0709,0.0859,0.1059,0.2009,4000],
                                    labels = ['Good', 'Moderate', 
                                              'Unhealthy for Sensitive Groups', "Unhealthy", 
                                              "Very Unhealthy", 'Hazardous'])
    # rename the new date time column
    average_24hr = average_24hr.rename(columns={"round_24hr": "datetime"})
    # Create air alerts
    def unhealthy_air_alert(c):
        if c['Pm2_5'] > 55.4:
            return 'Pm2_5'
        elif c['Pm10'] > 254.9:
            return 'Pm10'
        elif c['CO'] > 12.4:
            return 'CO'
        elif c['SO2'] > 0.1859:
            return 'SO2'
        elif c['O3'] > 0.1649:
            return 'O3'
        elif c['NO2'] > 0.3609:
            return 'NO2'
        else:
            return 'No Alert'
    average_24hr['unhealthy_alert'] = average_24hr.apply(unhealthy_air_alert, axis=1)
    # create sensitive HYPOTHTICAL Alert system
        # hypothetical alerts are based on IF everything is reading in PPM
    def sensitive_air_alert(c):
        if c['Pm2_5'] > 34.4:
            return 'Pm2_5'
        elif c['Pm10'] > 154.9:
            return 'Pm10'
        elif c['CO'] > 9.4:
            return 'CO'
        elif c['SO2'] > 0.0759:
            return 'SO2'
        elif c['O3'] > 0.124:
            return 'O3'
        elif c['NO2'] > 0.1009:
            return 'NO2'
        else:
            return 'No Alert'

    average_24hr['sensitive_alert'] = average_24hr.apply(sensitive_air_alert, axis=1)
    return average_24hr

average_24hr = air_24hr_avg(air)
average_24hr.head()

,datetime,Pm1_0,Pm2_5,Pm10,SO2,O3,CO,NO2,hour,weekday,...,Pm_25_24hr,Pm_10_24hr,AQI_CO,AQI_pm2_5,AQI_pm10,AQI_SO2,AQI_NO2,AQI_O3,unhealthy_alert,sensitive_alert
0,2021-04-20,5.467593,9.967593,8.000000,0.0,0.011574,0.342593,0.328704,5.500000,1.000000,...,8.022792,7.324786,Good,Good,Good,Good,Unhealthy for Sensitive Groups,Good,No Alert,NO2
1,2021-04-21,3.426554,4.005650,5.019774,0.0,0.002825,0.158192,0.138418,9.186441,1.618644,...,5.296852,5.552644,Good,Good,Good,Good,Unhealthy for Sensitive Groups,Good,No Alert,NO2
2,2021-04-22,9.888252,10.667622,13.524355,0.0,0.010315,0.111748,0.234957,9.229226,2.618911,...,12.671570,15.911842,Good,Good,Good,Good,Unhealthy for Sensitive Groups,Good,No Alert,NO2
3,2021-04-23,21.504298,22.922636,28.515759,0.0,0.008883,0.108883,0.320917,9.289398,3.610315,...,18.834655,23.545064,Good,Moderate,Good,Good,Unhealthy for Sensitive Groups,Good,No Alert,NO2
4,2021-04-24,7.495726,8.222222,56.279202,0.0,0.001994,0.316239,0.256410,9.179487,4.615385,...,10.132326,40.726284,Good,Good,Moderate,Good,Unhealthy for Sensitive Groups,Good,No Alert,NO2


In [23]:
air = wrangle.clean_air()

def air_1hr_avg(air):
    '''Takes in air df and creates every 8 hour averages'''
    # duplicate datetime column
    air['round_1hr'] = air['datetime']
    # round to every 8 hours
    air['round_1hr'] = air['round_1hr'].dt.round('60min')
    # create mew df based on the rouned time
    average_1hr = air.groupby('round_1hr', as_index=False).mean()
    # Create AQI for CO
    average_1hr['AQI_CO'] = pd.cut(average_1hr.CO, 
                                bins = [-1,4.5,9.5,12.5,15.5,30.5,4000],
                                labels = ['Good', 'Moderate', 
                                          'Unhealthy for Sensitive Groups', "Unhealthy", 
                                          "Very Unhealthy", 'Hazardous'])
    # Create AQI for Pm2_5
    average_1hr['AQI_pm2_5'] = pd.cut(average_1hr.Pm2_5, 
                                    bins = [-1,12.1,35.5,55.5,150.5,250.5,4000],
                                    labels = ['Good', 'Moderate', 
                                              'Unhealthy for Sensitive Groups', "Unhealthy", 
                                              "Very Unhealthy", 'Hazardous'])
    # Create AQI for Pm10
    average_1hr['AQI_pm10'] = pd.cut(average_1hr.Pm10, 
                                    bins = [-1,55,154,255,355,425,4000],
                                    labels = ['Good', 'Moderate', 
                                              'Unhealthy for Sensitive Groups', "Unhealthy", 
                                              "Very Unhealthy", 'Hazardous'])
    # create AQI for SO2
    average_1hr['AQI_SO2'] = pd.cut(average_1hr.SO2, 
                                    bins = [-1,0.0359,0.0759,0.1859,0.3049,0.6049,4000],
                                    labels = ['Good', 'Moderate', 
                                              'Unhealthy for Sensitive Groups', "Unhealthy", 
                                              "Very Unhealthy", 'Hazardous'])
    # create AQI for NO2
    average_1hr['AQI_NO2'] = pd.cut(average_1hr.NO2, 
                                    bins = [-1,0.0539,0.1009,0.3609,0.6499,1.2499,4000],
                                    labels = ['Good', 'Moderate', 
                                              'Unhealthy for Sensitive Groups', "Unhealthy", 
                                              "Very Unhealthy", 'Hazardous'])
    # create AQI for O3
    average_1hr['AQI_O3'] = pd.cut(average_1hr.O3, 
                                    bins = [-1,0.06259,0.1259,0.1649,0.2049,0.4049,4000],
                                    labels = ['Good', 'Moderate', 
                                              'Unhealthy for Sensitive Groups', "Unhealthy", 
                                              "Very Unhealthy", 'Hazardous'])
    # rename the new date time column
    average_1hr = average_1hr.rename(columns={"round_1hr": "datetime"})
    # Create air alerts
    def unhealthy_air_alert(c):
        if c['Pm2_5'] > 55.4:
            return 'Pm2_5'
        elif c['Pm10'] > 254.9:
            return 'Pm10'
        elif c['CO'] > 12.4:
            return 'CO'
        elif c['SO2'] > 0.1859:
            return 'SO2'
        elif c['O3'] > 0.1649:
            return 'O3'
        elif c['NO2'] > 0.3609:
            return 'NO2'
        else:
            return 'No Alert'
    average_1hr['unhealthy_alert'] = average_1hr.apply(unhealthy_air_alert, axis=1)
    # create sensitive HYPOTHTICAL Alert system
        # hypothetical alerts are based on IF everything is reading in PPM
    def sensitive_air_alert(c):
        if c['Pm2_5'] > 34.4:
            return 'Pm2_5'
        elif c['Pm10'] > 154.9:
            return 'Pm10'
        elif c['CO'] > 9.4:
            return 'CO'
        elif c['SO2'] > 0.0759:
            return 'SO2'
        elif c['O3'] > 0.124:
            return 'O3'
        elif c['NO2'] > 0.1009:
            return 'NO2'
        else:
            return 'No Alert'

    average_1hr['sensitive_alert'] = average_1hr.apply(sensitive_air_alert, axis=1)
    return average_1hr

average_1hr = air_1hr_avg(air)
average_1hr.head()

,datetime,Pm1_0,Pm2_5,Pm10,SO2,O3,CO,NO2,hour,weekday,...,Pm_25_24hr,Pm_10_24hr,AQI_CO,AQI_pm2_5,AQI_pm10,AQI_SO2,AQI_NO2,AQI_O3,unhealthy_alert,sensitive_alert
0,2021-04-20 00:00:00,6.777778,7.555556,9.777778,0.0,0.000000,0.000000,0.333333,0.0,1.0,...,8.022792,7.324786,Good,Good,Good,Good,Unhealthy for Sensitive Groups,Good,No Alert,NO2
1,2021-04-20 01:00:00,5.888889,6.888889,8.722222,0.0,0.000000,0.000000,0.333333,0.5,1.0,...,8.022792,7.324786,Good,Good,Good,Good,Unhealthy for Sensitive Groups,Good,No Alert,NO2
2,2021-04-20 02:00:00,7.888889,9.055556,11.500000,0.0,0.000000,0.000000,0.277778,1.5,1.0,...,8.022792,7.324786,Good,Good,Good,Good,Unhealthy for Sensitive Groups,Good,No Alert,NO2
3,2021-04-20 03:00:00,8.611111,9.555556,12.055556,0.0,0.000000,0.111111,0.277778,2.5,1.0,...,8.022792,7.324786,Good,Good,Good,Good,Unhealthy for Sensitive Groups,Good,No Alert,NO2
4,2021-04-20 04:00:00,7.888889,8.277778,10.388889,0.0,0.138889,0.277778,0.333333,3.5,1.0,...,8.022792,7.324786,Good,Good,Good,Good,Unhealthy for Sensitive Groups,Unhealthy for Sensitive Groups,No Alert,O3
